# Notebook settings

### Don't change these parameters :0

In [6]:
''' These parameters should not be changed!!!'''

# check device
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# pytorch weight initialization seed
RANDOM_SEED = 42

# model-specific image sizes
LENET5_IMG_SIZE = 32

### These parameters can be tampered with

In [7]:
'''These parameters can potentiall be changed...'''

LEARNING_RATE = 0.001
BATCH_SIZE = 32
N_EPOCHS = 15
N_CLASSES = 10

def update_globals(lr=0.001, bs=32, epochs=1, n_classes=260):
    '''Update specific parameters using this method'''
    LEARNING_RATE = lr
    BATCH_SIZE = bs
    N_EPOCHS = epochs
    N_CLASSES = n_classes


# Semi-supervised learning
We try and initialize the model weights by first training it on a different task: predicting image rotations on the unlabeled dataset that was given to us.

### Load Unlabeled Data
The 30,000 unlabeled samples have each been rotated 3 times (by 90, 180, and 270 degreeses respectively,) and merged into an array of 120,000 images and an array of 120,000 rotation labels (0, 1, 2, 3 corresponding to each rotation.)

In [8]:
import assignment_data as data

SPLIT_INDEX = 100000 # 100,000 training and 20,000 validation
x_train, x_val = data.split_train_and_val(data.rotated_images, split_index=SPLIT_INDEX)
y_train, y_val = data.split_train_and_val(data.rotated_image_labels, split_index=SPLIT_INDEX)

### Debugging vvv
print(f'x_train: {x_train.shape}, y_train: {y_train.shape}.\n' + 
f'x_val: {x_val.shape}, y_val: {y_val.shape}.')

x_train: (100000, 56, 56), y_train: (100000,).
x_val: (20000, 56, 56), y_val: (20000,).


### Create pytorch tensors, datasets, and dataloaders

In [9]:
# Make tensors out of the numpy arrays
x_train, y_train, x_val, y_val = map(torch.tensor, (x_train, y_train, x_val, y_val))

# Make TensorDatasets
from torch.utils.data import TensorDataset
train_ds = TensorDataset(x_train, y_train)
val_ds = TensorDataset(x_val, y_val)

# Make DataLoaders
from torch.utils.data import DataLoader
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

### Train LeNet5 on the Image Rotation Task

In [10]:
from model_definitions import LeNet5
from torch import nn
from model_training import training_loop

def downsample(batch, size=32):
    '''Downsamples the data to fit model requirements. I couldn't find a way around this that worked effectively...'''
    batch = batch.view(-1, 1, 56, 56) # Resize input to make it square
    downsample = nn.AdaptiveAvgPool2d(size) # Define a downsampling function
    batch = downsample(batch) # Downsample the batch's image data
    return batch

lenet5 = LeNet5(N_CLASSES, preprocess=downsample) # Load LeNet architecture
optimizer = torch.optim.Adam(lenet5.parameters(), lr=LEARNING_RATE) # Load optimizer
criterion = nn.CrossEntropyLoss() # Specifiy loss function
model, optimizer, _ = training_loop(lenet5, criterion, optimizer, train_dl, val_dl, N_EPOCHS, DEVICE)

### Load assignment data

In [ ]:
import assignment_data as data
from torch.utils.data import TensorDataset

# Dataset variables
VAL_SPLIT = 25000

# Partition labeled data into train and val datasets
X_tmp = data.labeled_images#.reshape(-1, 3136)
y_tmp = Data.integer_labels() # Binary to number/letter, then number/letter to unique integer represntation for each label (conv net is only happy with 1D numerical outputs)
X_val, y_val = X_tmp[VAL_SPLIT:], y_tmp[VAL_SPLIT:]
X_train, y_train = X_tmp[0:VAL_SPLIT], y_tmp[0:VAL_SPLIT]

print(f'X_train: {X_train.shape} \ny_train: {y_train.shape}')
print(f'X_val: {X_val.shape} \ny_val: {y_val.shape}')

# Transform the labeled train and val datasets into pytorch Tensors
X_train, y_train, X_val, y_val = map(
    torch.tensor, (X_train, y_train, X_val, y_val)
)

# Initialize pytorch TensorDatasets
train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)

# Initialize pytorch DataLoaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True) # Shuffled to prevent correlation betwen batch ordering and model overfitting, see: https://pytorch.org/tutorials/beginner/nn_tutorial.html#add-validation
valid_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

ModuleNotFoundError: No module named 'Data'

### Train model on project dataset

In [ ]:
torch.manual_seed(RANDOM_SEED)

# Set number of classes
N_CLASSES = 260 # One for each letter-number combo

def preprocess_batch(batch):
    batch = batch.view(-1, 1, 56, 56) # Resize input to make it square
    downsample = nn.AdaptiveAvgPool2d((32, 32)) # Define a downsampling function
    batch = downsample(batch) # Downsample the batch's image data
    return batch

model = LeNet5(N_CLASSES, preprocess=preprocess_batch).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [ ]:
model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS, DEVICE)

KeyboardInterrupt: 